<font size="18">With Bubbles, No Troubles</font>

In [ ]:
# set this notebook to use a large part of the browser window width
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:80% !important; }</style>"))

# Introduction

We consider the problem of allocating all people p in a set P a spot (x,y), where
... see README.txt i  this dir.

# Model Setup



In [ ]:
from IPython.display import Image
#Image(filename='ChainGangOnAFence.png', width=1000, height=3000)

Using Kirchhoff current and voltege laws we get the equations.

In [ ]:
from gurobipy import *
import random

random.seed()

program = 'WithBubblesNoTroubles'

# generate some bubbles with people:
max_persons_per_bubble = 5
B = 4

p = 0
bubbles = []
persons = []
for b in range(0, B):
    bubble_b_size = random.randint(1, max_persons_per_bubble)
    for i in range(0, bubble_b_size):
        bubbles.append(b)
        persons.append(p)        
        p += 1

P = len(persons)
print('P = {:d}'.format(P))
print('persons = ')
print(persons)

print('B = {:d}'.format(B))
print('bubbles = ')
print(bubbles)

dSameBubbleMLo = 0.3 # meter
dSameBubbleMHi = 3.0 # meter

dOtherBMLo = 1.5 # meter

def model(xlo=0, xhi=100,
          ylo=0, yhi=100):
    
    verbose = 0
        
    m = Model()

    # create all the 4N continuous variables:
    x = m.addVars(persons, lb=[xlo]*P, ub=[xhi]*P, vtype=GRB.CONTINUOUS, name='x')  
    y = m.addVars(persons, lb=[ylo]*P, ub=[yhi]*P, vtype=GRB.CONTINUOUS, name='y')  

    assert len(x) == P
    assert len(y) == P

    # (1) create all the 3N linear current constraints:
    for p1 in persons:  # there are P of them: 0..P-1
        for p2 in persons:  # there are P of them: 0..P-1
            if p1 < p2:
                quad_expr = (x[p2] - x[p1]) *  (x[p2] - x[p1]) + (y[p2] - y[p1]) * (y[p2] - y[p1])
                if bubbles[p1] == bubbles[p2]:  # pair is in same bubble, pull together
                    m.addConstr(quad_expr >= dSameBubbleMLo**2, 'sameMLo_p{:d}_p{:d}'.format(p1,p2))
                    m.addConstr(quad_expr <= dSameBubbleMHi**2, 'sameMHi_p{:d}_p{:d}'.format(p1,p2))
                else: # pair is not in same bubble, separate by pushing away from each other
                    m.addConstr(quad_expr >= dOtherBMLo**2, 'otherMLo_p{:d}_p{:d}'.format(p1,p2))

    m.setParam(GRB.Param.NonConvex, 2)
    m.write(program + '.lp')
    m.optimize()
    print(x)
    print(y)
    return m, x, y
    
m, x, y = model()

In [ ]:
# Select B different colors to display the bubbles in different colors,
# so that the use can immediately see that bubbles are well separated.
import numpy as np
import colorsys

def _get_colors(num_colors):
    colors=[]
    for i in np.arange(0., 360., 360. / num_colors):
        hue = i/360.
        lightness = (50 + np.random.rand() * 10)/100.
        saturation = (90 + np.random.rand() * 10)/100.
        colors.append(colorsys.hls_to_rgb(hue, lightness, saturation))
    return colors

bubble_to_color_map = _get_colors(B)

In [ ]:
#import numpy as np

def print_and_convert_results(m, x, y):     
    for var_name, var_struct in list(zip(['x', 'y'], [x, y])):
        values = m.getAttr('x', var_struct)
        s = var_name
        s += ' (m) = '
        vals = [(values[v]) for v in values]
        do_print = False
        if do_print:
            print(s)
            print(vals)
        
        if var_name == 'x':
            x_values = vals
            if do_print:
                print(x_values)
        else:
            y_values = vals
            if do_print:
                print(y_values)
    return x_values, y_values

    
x_values, y_values = print_and_convert_results(m, x, y)


import matplotlib


import pandas as pd
df = pd.DataFrame(columns=[ "person", "bubble", "x", "y", "color"])
df["person"] = persons
df["bubble"] = bubbles
df["color"] = [matplotlib.colors.to_hex(bubble_to_color_map[b]) for b in bubbles]
df["x"] = x_values
df["y"] = y_values

df

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df["x"], df["y"], color=df["color"], alpha=0.7)